# Compute the price for all LEGO Bricks of a set

In [1]:
import re
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool

In [2]:
LEGO_Set = '79003-1'

url_parts = 'https://www.bricklink.com/catalogItemInv.asp?S={}'.format(LEGO_Set)
url_minifigs = 'https://www.bricklink.com/catalogItemInv.asp?S={}&viewItemType=M'.format(LEGO_Set)

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [3]:
data = []

bs = BeautifulSoup(requests.get(url_parts, headers=headers).text, 'lxml')
for elem in bs.find('table', {'class':'ta'}).findAll('tr'):
    try:
        tds = elem.findAll('td')
        quantity = int(tds[1].get_text())
        url = tds[2].find('a')['href']
        data.append([quantity, url])
    except:
        pass

bs = BeautifulSoup(requests.get(url_minifigs, headers=headers).text, 'lxml')
for elem in bs.find('table', {'class':'ta'}).findAll('tr'):
    try:
        tds = elem.findAll('td')
        quantity = int(tds[1].get_text())
        url = tds[2].find('a')['href']
        data.append([quantity, url])
    except:
        pass
    
df = pd.DataFrame(data, columns=['Quantity', 'URL'])
df.head(2)

,Quantity,URL
0,1,/v2/catalog/catalogitem.page?P=79003stk01&idCo...
1,1,/v2/catalog/catalogitem.page?P=3659&idColor=11


In [4]:
re_part = re.compile('P=(.+)&')
re_color = re.compile('idColor=(\d+)')

In [5]:
def scrapeBricks(link):
    try:
        part = re.search(re_part, link)[1]
        color = re.search(re_color, link)[1]
        
        url = url_price.format(part, color)
    except:
        part = 'missing'
        color = 'missing'
        url = 'https://www.bricklink.com{}'.format(link)
    
    price = 0
    bs = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')
    try:
        for elem in bs.find('table', {'class':'fv'}).findAll('tr'):
            text = elem.get_text()
            s = re.search('Qty Avg Price:EUR(.+?)Max', text)
            if s:
                price = float(s[1].strip())
                break
    except:
        pass
    return [part, color, price, link]

In [6]:
url_price = 'https://www.bricklink.com/catalogPG.asp?P={}&ColorID={}'

data = []
futures = []
pool = Pool(2)

for link in df['URL'].values:
    futures.append(pool.apply_async(scrapeBricks, [link]))
for brick in tqdm(futures):
    data.append(brick.get())

100%|██████████| 228/228 [08:09<00:00,  1.82it/s]


In [7]:
df_parts = pd.DataFrame(data, columns=['Part', 'Color', 'Price', 'URL'])
df = df.merge(df_parts, on='URL', how='outer')

In [8]:
df

,Quantity,URL,Part,Color,Price
0,1,/v2/catalog/catalogitem.page?P=79003stk01&idCo...,79003stk01,0,1.99
1,1,/v2/catalog/catalogitem.page?P=3659&idColor=11,3659,11,0.05
2,1,/v2/catalog/catalogitem.page?P=3633&idColor=11,3633,11,0.08
3,1,/v2/catalog/catalogitem.page?P=6134&idColor=11,6134,11,0.05
4,1,/v2/catalog/catalogitem.page?P=4332&idColor=11,4332,11,0.16
5,1,/v2/catalog/catalogitem.page?P=4528&idColor=11,4528,11,0.07
6,2,/v2/catalog/catalogitem.page?P=4529&idColor=11,4529,11,0.06
7,1,/v2/catalog/catalogitem.page?P=4864b&idColor=11,4864b,11,0.14
8,1,/v2/catalog/catalogitem.page?P=3024&idColor=11,3024,11,0.05
9,1,/v2/catalog/catalogitem.page?P=3024&idColor=11,3024,11,0.05


In [9]:
df['All_Price'] = df['Quantity']*df['Price']

In [10]:
df.sort_values('All_Price', ascending=False)

,Quantity,URL,Part,Color,Price,All_Price
62,2,/v2/catalog/catalogitem.page?P=92438&idColor=69,92438,69,1.20,2.40
266,1,/v2/catalog/catalogitem.page?P=33009pb039&idCo...,33009pb039,88,2.17,2.17
0,1,/v2/catalog/catalogitem.page?P=79003stk01&idCo...,79003stk01,0,1.99,1.99
161,25,/v2/catalog/catalogitem.page?P=3062b&idColor=88,3062b,88,0.05,1.25
99,1,/v2/catalog/catalogitem.page?P=92438&idColor=6,92438,6,1.16,1.16
202,5,/v2/catalog/catalogitem.page?P=3008&idColor=2,3008,2,0.23,1.15
160,14,/v2/catalog/catalogitem.page?P=4070&idColor=88,4070,88,0.08,1.12
139,14,/v2/catalog/catalogitem.page?P=3069b&idColor=150,3069b,150,0.08,1.12
135,10,/v2/catalog/catalogitem.page?P=98283&idColor=150,98283,150,0.11,1.10
201,7,/v2/catalog/catalogitem.page?P=3009&idColor=2,3009,2,0.15,1.05


In [11]:
df['All_Price'].sum()

68.4899999999999